In [1]:
import pandas as pd
import numpy as np
import sklearn
import matplotlib.pyplot as plt
import joblib
import os
import shutil
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split, GridSearchCV, learning_curve
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge
from sklearn.pipeline import Pipeline


In [2]:
df = pd.read_csv("PastureCastModel_V1.1.1.csv")
df

,Experiment,Date,season,Plot,Strip,MeanHeight(mm),Biomass(kg/ha),SAVI_mean,EVI_mean,NDVI_mean,...,CLRE_mean,SRre_mean,Cum_21D_Precip_in,Cum_21D_GDD,Avg_21D_SWB_in,Avg_21D_SWB_pct,Cum_21D_ETo_in,Avg_21D_Temp_C,Growth_rt_21D,GR_SWB_21D
0,TB_2024,9/18/2024,Fall,14.1,1,105,2277,0.43,0.44,0.59,...,1.47,2.47,0.16,820,0.04,0.01,4.23,22,0.99,0.00
1,TB_2024,9/18/2024,Fall,14.1,3,149,3006,0.44,0.47,0.61,...,1.60,2.60,0.16,820,0.04,0.01,4.23,22,0.99,0.00
2,TB_2024,9/18/2024,Fall,14.1,7,120,3195,0.44,0.47,0.61,...,1.51,2.51,0.16,820,0.04,0.01,4.23,22,0.99,0.00
3,TB_2024,9/18/2024,Fall,14.1,11,111,2597,0.44,0.47,0.61,...,1.56,2.56,0.16,820,0.04,0.01,4.23,22,0.99,0.00
4,TB_2024,9/18/2024,Fall,14.1,12,100,2598,0.43,0.44,0.59,...,1.47,2.47,0.16,820,0.04,0.01,4.23,22,0.99,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
969,TB_2025,6/25/2025,Spring,28.4,18,277,4235,0.62,0.68,0.80,...,2.44,3.44,6.70,833,3.40,0.71,3.89,22,0.99,0.71
970,TB_2025,6/25/2025,Spring,28.4,19,260,4232,0.61,0.67,0.79,...,2.32,3.32,6.70,833,3.40,0.71,3.89,22,0.99,0.71
971,TB_2025,6/25/2025,Spring,28.4,20,267,4566,0.61,0.67,0.79,...,2.27,3.27,6.70,833,3.40,0.71,3.89,22,0.99,0.71
972,TB_2025,6/25/2025,Spring,28.4,21,96,1966,0.61,0.67,0.79,...,2.27,3.27,6.70,833,3.40,0.71,3.89,22,0.99,0.71


In [3]:
df.dtypes

Experiment            object
Date                  object
season                object
Plot                 float64
Strip                  int64
MeanHeight(mm)         int64
Biomass(kg/ha)         int64
SAVI_mean            float64
EVI_mean             float64
NDVI_mean            float64
MSAVI_mean           float64
GNDVI_mean           float64
NDRE_mean            float64
CLRE_mean            float64
SRre_mean            float64
Cum_21D_Precip_in    float64
Cum_21D_GDD            int64
Avg_21D_SWB_in       float64
Avg_21D_SWB_pct      float64
Cum_21D_ETo_in       float64
Avg_21D_Temp_C         int64
Growth_rt_21D        float64
GR_SWB_21D           float64
dtype: object

In [4]:
df.isnull().sum()

Experiment           0
Date                 0
season               0
Plot                 0
Strip                0
MeanHeight(mm)       0
Biomass(kg/ha)       0
SAVI_mean            0
EVI_mean             0
NDVI_mean            0
MSAVI_mean           0
GNDVI_mean           0
NDRE_mean            0
CLRE_mean            0
SRre_mean            0
Cum_21D_Precip_in    0
Cum_21D_GDD          0
Avg_21D_SWB_in       0
Avg_21D_SWB_pct      0
Cum_21D_ETo_in       0
Avg_21D_Temp_C       0
Growth_rt_21D        0
GR_SWB_21D           0
dtype: int64

In [5]:
# Ensure the 'Date' column is in datetime format (handles mixed formats)
df['Date'] = pd.to_datetime(df['Date'], format='mixed')
df['Julian Date'] = df['Date'].dt.dayofyear
df

,Experiment,Date,season,Plot,Strip,MeanHeight(mm),Biomass(kg/ha),SAVI_mean,EVI_mean,NDVI_mean,...,SRre_mean,Cum_21D_Precip_in,Cum_21D_GDD,Avg_21D_SWB_in,Avg_21D_SWB_pct,Cum_21D_ETo_in,Avg_21D_Temp_C,Growth_rt_21D,GR_SWB_21D,Julian Date
0,TB_2024,2024-09-18,Fall,14.1,1,105,2277,0.43,0.44,0.59,...,2.47,0.16,820,0.04,0.01,4.23,22,0.99,0.00,262
1,TB_2024,2024-09-18,Fall,14.1,3,149,3006,0.44,0.47,0.61,...,2.60,0.16,820,0.04,0.01,4.23,22,0.99,0.00,262
2,TB_2024,2024-09-18,Fall,14.1,7,120,3195,0.44,0.47,0.61,...,2.51,0.16,820,0.04,0.01,4.23,22,0.99,0.00,262
3,TB_2024,2024-09-18,Fall,14.1,11,111,2597,0.44,0.47,0.61,...,2.56,0.16,820,0.04,0.01,4.23,22,0.99,0.00,262
4,TB_2024,2024-09-18,Fall,14.1,12,100,2598,0.43,0.44,0.59,...,2.47,0.16,820,0.04,0.01,4.23,22,0.99,0.00,262
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
969,TB_2025,2025-06-25,Spring,28.4,18,277,4235,0.62,0.68,0.80,...,3.44,6.70,833,3.40,0.71,3.89,22,0.99,0.71,176
970,TB_2025,2025-06-25,Spring,28.4,19,260,4232,0.61,0.67,0.79,...,3.32,6.70,833,3.40,0.71,3.89,22,0.99,0.71,176
971,TB_2025,2025-06-25,Spring,28.4,20,267,4566,0.61,0.67,0.79,...,3.27,6.70,833,3.40,0.71,3.89,22,0.99,0.71,176
972,TB_2025,2025-06-25,Spring,28.4,21,96,1966,0.61,0.67,0.79,...,3.27,6.70,833,3.40,0.71,3.89,22,0.99,0.71,176


In [6]:
# 1️⃣ Define the function
def julian_cos_encoding_from_date(df: pd.DataFrame, date_col: str, cos_JD: str = "Julian_Cos") -> pd.DataFrame:
    """
    Takes a real date column, extracts year and day-of-year,
    checks leap year, converts to radians, then applies cosine conversion.
    """
    df[date_col] = pd.to_datetime(df[date_col])
    years = df[date_col].dt.year
    julian_days = df[date_col].dt.dayofyear
    days_in_year = np.where(
        ((years % 4 == 0) & (years % 100 != 0)) | (years % 400 == 0),
        366, 365
    )
    radians = 2 * np.pi * julian_days / days_in_year
    df[cos_JD] = np.cos(radians)
    return df

# 2️⃣ Apply the function BEFORE defining your features
df = julian_cos_encoding_from_date(df, date_col="Date")


In [7]:
df.columns

Index(['Experiment', 'Date', 'season', 'Plot', 'Strip', 'MeanHeight(mm)',
       'Biomass(kg/ha)', 'SAVI_mean', 'EVI_mean', 'NDVI_mean', 'MSAVI_mean',
       'GNDVI_mean', 'NDRE_mean', 'CLRE_mean', 'SRre_mean',
       'Cum_21D_Precip_in', 'Cum_21D_GDD', 'Avg_21D_SWB_in', 'Avg_21D_SWB_pct',
       'Cum_21D_ETo_in', 'Avg_21D_Temp_C', 'Growth_rt_21D', 'GR_SWB_21D',
       'Julian Date', 'Julian_Cos'],
      dtype='object')

In [12]:
# Define the independent variables (features) and the target variable
features = ['MeanHeight(mm)', 'Julian_Cos',
            'SAVI_mean', 'EVI_mean', 'NDVI_mean', 'NDRE_mean', 'Avg_21D_SWB_pct',]
target = 'Biomass(kg/ha)'

# Ensure your data is clean and handle missing values
X = df[features]
y = df[target]

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

# Preprocessing pipeline - Include all your preprocessing steps in below function to be applied to new data
preprocessor = Pipeline(steps=[
    ('scaler', StandardScaler())
])

# have variable names as shown below _preprocessed
# Apply preprocessing
X_train_preprocessed = preprocessor.fit_transform(X_train)
X_test_preprocessed = preprocessor.transform(X_test)

# --- Ridge regression and hyperparameter tuning ---
param_grid = {'alpha': np.logspace(-2, 2, 10)}  # Exploring different alpha values
ridge = Ridge()

# Perform Grid Search with Cross-Validation
grid_search = GridSearchCV(ridge, param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X_train_preprocessed, y_train)

# Always register your model into variable named "model"
model = grid_search.best_estimator_

# Predict on test set
y_pred = model.predict(X_test_preprocessed)

# Evaluation Metrics (using absolute predictions)
test_mse = mean_squared_error(y_test, y_pred)
test_rmse = test_mse ** 0.5 
test_mae = mean_absolute_error(y_test, y_pred)  # MAE
test_r2 = r2_score(y_test, y_pred)  # R² Score

# Print evaluation results
print(f"Best Alpha: {grid_search.best_params_['alpha']}")
print(f"Test MSE: {test_mse:.4f}")
print(f"Test RMSE: {test_rmse:.4f}")
print(f"Test MAE: {test_mae:.4f}")
print(f"Test R² Score: {test_r2:.4f}")

# --- Print Ridge regression equation with feature names ---
feature_names = X_train.columns  # assumes X_train is a DataFrame
coefs = model.coef_
intercept = model.intercept_

eq_terms = [f"({coef:.3f} * {name})" for coef, name in zip(coefs, feature_names)]
equation = " + ".join(eq_terms)
equation = f"y = {intercept:.3f} + {equation}"

print("\nRidge Regression Equation with Feature Names:")
print(equation)

Best Alpha: 0.01
Test MSE: 202841.5920
Test RMSE: 450.3794
Test MAE: 320.8801
Test R² Score: 0.9339

Ridge Regression Equation with Feature Names:
y = 3132.018 + (1940.558 * MeanHeight(mm)) + (389.131 * Julian_Cos) + (-953.665 * SAVI_mean) + (929.615 * EVI_mean) + (-164.584 * NDVI_mean) + (92.121 * NDRE_mean) + (361.667 * Avg_21D_SWB_pct)


In [ ]:

# Define the path ----- Change Accodingly
save_path = r"Biomass Lab work\Testbed_2025TB_Model_V1.1.1\PastureCastModel_V1.1.1_Testing"

# Make sure the folder exists (creates it if it doesn't)
os.makedirs(save_path, exist_ok=True)

# File names
model_file = os.path.join(save_path, "ridge_model.joblib")
scaler_file = os.path.join(save_path, "scaler.joblib")

# Save model and scaler
joblib.dump(best_ridge, model_file)
joblib.dump(scaler, scaler_file)

print(f"Trained Ridge model saved to {model_file}")
print(f"Scaler saved to {scaler_file}")


In [ ]:

shutil.copy(os.path.join(save_path, "ridge_model.joblib"), "./ridge_model.joblib")
shutil.copy(os.path.join(save_path, "scaler.joblib"), "./scaler.joblib")
